<a href="https://colab.research.google.com/github/dragssvd/UN_Image_Inpainting_Project/blob/main/ENC_DEC_CLASS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [6]:
!pip install comet-ml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.1/698.1 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 969.6/969.6 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 8.3 MB/s eta 0:00:00
  Created wheel for configobj: filename=configobj-5.0.9-py2.py3-none-any.whl size=35614 sha256=d54591abded4c1fdc8f8f3626b675e4ff5b8a698750d1e8ec14d3af9041e8489
  Stored in directory: /root/.cache/pip/wheels/a1/6c/03/6c5e3cf1a6e4b9e2fc5c4409be4abc5a8268bd9c878739cb32
Successfully built configobj
  Attempting uninstall: python-box
    Found existing installation: python-box 7.2.0
    Uninstalling python-box-7.2.0:
      Successfully uninstalled python-box-7.2.0


In [7]:
import comet_ml

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import einops
import numpy as np

import matplotlib
from matplotlib import pyplot as plt

## CometML

In [8]:
import os

from google.colab import userdata
key = userdata.get('COMET_API_KEY')
os.environ['COMET_API_KEY'] = key

## Pick device

In [9]:
def setup_device():
    if torch.cuda.is_available():
        device = torch.device('cuda')
        # Set default tensor type for cuda
        torch.set_default_dtype(torch.float32)
    elif torch.backends.mps.is_available():
        device = torch.device('mps')
    else:
        device = torch.device('cpu')
        # Ensure we're using float32 on CPU
        torch.set_default_dtype(torch.float64)
    return device

device = setup_device()

print(f"Using {device} device")

Using cpu device


# Dataset

In [10]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [11]:
from datasets import load_dataset, DatasetDict

ds = load_dataset("Artificio/WikiArt_Full").with_format("torch")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/780 [00:00<?, ?B/s]

(…)-00000-of-00007-dd79acf31a667474.parquet:   0%|          | 0.00/462M [00:00<?, ?B/s]

(…)-00001-of-00007-16d737d74ca9cb8a.parquet:   0%|          | 0.00/463M [00:00<?, ?B/s]

(…)-00002-of-00007-73e15989674f270b.parquet:   0%|          | 0.00/464M [00:00<?, ?B/s]

(…)-00003-of-00007-911b21abee70c83e.parquet:   0%|          | 0.00/464M [00:00<?, ?B/s]

(…)-00004-of-00007-a49c31eac1929942.parquet:   0%|          | 0.00/464M [00:00<?, ?B/s]

(…)-00005-of-00007-053e48131b243ec0.parquet:   0%|          | 0.00/465M [00:00<?, ?B/s]

(…)-00006-of-00007-daeb2ac40cedaee0.parquet:   0%|          | 0.00/464M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/103250 [00:00<?, ? examples/s]

In [12]:
ds

DatasetDict({
    train: Dataset({
        features: ['title', 'artist', 'date', 'genre', 'style', 'description', 'filename', 'image', 'embeddings_pca512', 'resnet50_non_robust_feats', 'resnet50_robust_feats'],
        num_rows: 103250
    })
})

In [13]:
# 90% train, 10% test + validation
train_testvalid = ds['train'].train_test_split(test_size=0.2)
# Split the 10% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
# gather everyone if you want to have a single DatasetDict
train_test_valid_dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']}).with_format("torch")

ds = train_test_valid_dataset
print(ds)

DatasetDict({
    train: Dataset({
        features: ['title', 'artist', 'date', 'genre', 'style', 'description', 'filename', 'image', 'embeddings_pca512', 'resnet50_non_robust_feats', 'resnet50_robust_feats'],
        num_rows: 82600
    })
    test: Dataset({
        features: ['title', 'artist', 'date', 'genre', 'style', 'description', 'filename', 'image', 'embeddings_pca512', 'resnet50_non_robust_feats', 'resnet50_robust_feats'],
        num_rows: 10325
    })
    valid: Dataset({
        features: ['title', 'artist', 'date', 'genre', 'style', 'description', 'filename', 'image', 'embeddings_pca512', 'resnet50_non_robust_feats', 'resnet50_robust_feats'],
        num_rows: 10325
    })
})


In [28]:
# Extract 10% of the train set
ten_percent_train = ds["train"].select(range(int(len(ds["train"]) * 0.1)))

# Extract 10% of the test set
ten_percent_test = ds["test"].select(range(int(len(ds["test"]) * 0.1)))

# Extract 10% of the validation set
ten_percent_valid = ds["valid"].select(range(int(len(ds["valid"]) * 0.1)))

# Combine the subsets into a new DatasetDict
ten_percent_dataset = DatasetDict({
    "train": ten_percent_train,
    "test": ten_percent_test,
    "valid": ten_percent_valid
})

ds = ten_percent_dataset
# for testing

In [14]:
# ds["test"]["image"][42]

In [15]:
# ds["test"]["style"][42]

In [16]:
# # Assuming 'ds' is your DatasetDict
# image = ds["test"]["image"][42]
# # Convert PIL.Image.Image to NumPy array
# image_np = np.array(image)
# # Transpose the dimensions
# plt.imshow(image_np.transpose(1, 2, 0))
# plt.show()

## Dataloader


In [29]:
  # change if not colab
  num_workers = 2
  pin_memory = True

  train_loader = DataLoader(
      ds["train"],
      batch_size=32,
      num_workers=num_workers,
      shuffle=True,
      pin_memory=True,
  )

  test_loader = DataLoader(
      ds["test"],
      batch_size=32,
      num_workers=num_workers,
      shuffle=False,
      pin_memory=True,
  )

  val_loader = DataLoader(
      ds["test"],
      batch_size=32,
      num_workers=num_workers,
      shuffle=False,
      pin_memory=True,
  )

# Autoencoder


## **Encoders**

### Basic encoder

In [30]:
import torch
import torch.nn as nn
import einops

class Encoder(nn.Module):
    @staticmethod
    def conv_block(in_size: int, out_size: int):
        return nn.Sequential(
            nn.Conv2d(in_size, out_size, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(out_size),
            nn.LeakyReLU(),
        )

    def __init__(self, latent_width: int = 64, **kwargs) -> None:
        super().__init__()
        self.latent_width = latent_width

        self.model = nn.Sequential(
            Encoder.conv_block(3, 8),   # [32, 8, 128, 128]
            Encoder.conv_block(8, 8),   # [32, 8, 64, 64]
            Encoder.conv_block(8, 16),  # [32, 16, 32, 32]
            Encoder.conv_block(16, 16), # [32, 16, 16, 16]
            Encoder.conv_block(16, 16), # [32, 16, 8, 8]
            Encoder.conv_block(16, 16), # [32, 16, 4, 4]
            Encoder.conv_block(16, 16), # [32, 16, 2, 2]
        )

        # Adjust Linear layer input to match 16 * 2 * 2
        self.fc = nn.Sequential(
            nn.Linear(16 * 2 * 2, self.latent_width),
            nn.Tanh(),
        )

    def forward(self, x: torch.Tensor):
        x = self.model(x)  # Convolutional layers
        x = einops.rearrange(x, "b c w h -> b (c w h)")  # Flatten for Linear
        x = self.fc(x)  # Fully connected layers
        return x

### Variance Encoder

In [31]:
class VarianceEncoder(nn.Module):
    @staticmethod
    def conv_block(in_size: int, out_size: int):
        return nn.Sequential(
            nn.Conv2d(in_size, out_size, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(out_size),
            nn.LeakyReLU(),
        )

    def __init__(self, latent_width: int = 64, **kwargs) -> None:
        super().__init__()
        self.latent_width = latent_width

        self.model = nn.Sequential(
            Encoder.conv_block(3, 8),
            Encoder.conv_block(8, 8),
            Encoder.conv_block(8, 16),
            Encoder.conv_block(16, 16),
        )

        self.fc_mu = nn.Linear(16 * 2 * 2, self.latent_width)
        self.fc_logvar = nn.Linear(16 * 2 * 2, self.latent_width)

    def reparametrize(self, mean: torch.Tensor, logvariance: torch.Tensor):
        # transform N(0,1) into N(mean, variance)
        std_dev = torch.exp(0.5 * logvariance)
        eps = torch.randn_like(std_dev)
        return eps * std_dev + mean

    def encode(self, x: torch.Tensor):
        x = self.model(x)
        x = einops.rearrange(x, "b c w h -> b (c w h)")
        mean = self.fc_mu(x)
        logvariance = self.fc_logvar(x)
        return mean, logvariance

    def forward(self, x: torch.Tensor):
        mean, logvariance = self.encode(x)
        z = self.reparametrize(mean, logvariance)
        return z, mean, logvariance

### Loss function for variance encoder

In [32]:
def kld_loss_func(mean, logvariance):
    # distance from normal distribution

    # commonly seen version:
    # return torch.mean(
    #     -0.5 * torch.sum(1 + logvariance - mean**2 - logvariance.exp(), dim=1),
    #     dim=0,
    # )

    loss = einops.reduce(
        1 + logvariance - mean**2 - torch.exp(logvariance),
        "batch latent -> batch",
        "sum",
    )
    return einops.reduce(-0.5 * loss, "batch -> ()", "mean")


## Decoder

In [33]:
import torch
import torch.nn as nn
import einops

class Decoder(nn.Module):
    @staticmethod
    def conv_block(in_size: int, out_size: int):
        return nn.Sequential(
            nn.ConvTranspose2d(
                in_size,
                out_size,
                kernel_size=3,
                stride=2,
                padding=1,
                output_padding=1,  # Ensures alignment of dimensions
            ),
            nn.BatchNorm2d(out_size),
            nn.LeakyReLU(),
        )

    def __init__(self, latent_width: int = 64, **kwargs) -> None:
        super().__init__()
        self.latent_width = latent_width

        # Fully connected layer to project latent vector into feature map
        self.fc = nn.Sequential(
            nn.Linear(self.latent_width, 16 * 2 * 2),
            nn.ReLU(),
        )

        # Transposed convolutions to upsample
        self.model = nn.Sequential(
            Decoder.conv_block(16, 16),  # [16, 2, 2] -> [16, 4, 4]
            Decoder.conv_block(16, 16),  # [16, 4, 4] -> [16, 8, 8]
            Decoder.conv_block(16, 16),  # [16, 8, 8] -> [16, 16, 16]
            Decoder.conv_block(16, 8),   # [16, 16, 16] -> [8, 32, 32]
            Decoder.conv_block(8, 8),    # [8, 32, 32] -> [8, 64, 64]
            Decoder.conv_block(8, 8),    # [8, 64, 64] -> [8, 128, 128]
            Decoder.conv_block(8, 3),    # [8, 128, 128] -> [3, 256, 256]
        )

        # Final activation ensures output in [0, 1] range
        self.final_activation = nn.Sigmoid()

    def forward(self, x: torch.Tensor):
        # Fully connected layer to expand latent vector
        x = self.fc(x)
        x = einops.rearrange(x, "b (c w h) -> b c w h", c=16, w=2, h=2)

        # Transposed convolutions to reconstruct image
        x = self.model(x)

        # Final activation for output normalization
        x = self.final_activation(x)
        return x

## Autoencoder

In [34]:
class Autoencoder(nn.Module):
    def __init__(
        self,
        encoder: Encoder,
        decoder: Decoder,
        *args,
        **kwargs,
    ) -> None:
        super().__init__(*args, **kwargs)
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, images: torch.Tensor):
        latent = self.encoder(images)
        decoded = self.decoder(latent)

        return decoded, latent

# Model setup

In [35]:
# model
latent_width = 64
encoder = Encoder(latent_width)
decoder = Decoder(encoder.latent_width)
model = Autoencoder(encoder, decoder)
model.to(device)

Autoencoder(
  (encoder): Encoder(
    (model): Sequential(
      (0): Sequential(
        (0): Conv2d(3, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.01)
      )
      (1): Sequential(
        (0): Conv2d(8, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.01)
      )
      (2): Sequential(
        (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.01)
      )
      (3): Sequential(
        (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2)

## Comet_ML setup logger

In [24]:
!pip install torchinfo
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 14.2 MB/s eta 0:00:00


In [26]:
# logging

from torch.utils.data import DataLoader  # NOQA
from datasets import load_dataset  # NOQA

from torchinfo import summary  # NOQA

from tqdm import tqdm  # NOQA

from comet_ml.integration.pytorch import log_model  # NOQA

from torchmetrics.functional.image import structural_similarity_index_measure  # NOQA

comet_experiment = comet_ml.Experiment(
    project_name="UczenieNienadzorowane")
comet_experiment.log_code(folder="/UN")
comet_experiment.log_parameters(
    {
        "batch_size": train_loader.batch_size,
        "train_size": ds["train"].num_rows,
        "val_size": ds["test"].num_rows,
    }
)

summ = summary(model, (1, 3, 256, 256), device=device, depth=5)
comet_experiment.set_model_graph(f"{model.__repr__()}\n{summ}")


COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : coloured_turtle_4859
COMET INFO:     url                   : https://www.comet.com/dragssvd/uczenienienadzorowane/2b3bb40a618e434aa1ff202f127d6779
COMET INFO:   Others:
COMET INFO:     notebook_url : https://colab.research.google.com/notebook#fileId=1Lm6PPTFOKM4e7twI4oBN58hDmkMQztBW
COMET INFO:   Parameters:
COMET INFO:     batch_size : 32
COMET INFO:     train_size : 82600
COMET INFO:     val_size   : 10325
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     installed packages  :

RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: []

# Train autoencoder

In [36]:
# pure PyTorch loop
num_epochs = 5
# loss_func = F.mse_loss
loss_func = F.l1_loss
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


In [96]:
comet_experiment.log_parameter("num_epochs", num_epochs)
comet_experiment.log_parameter("loss_func", loss_func.__name__)

comet_experiment.add_tag(f"LOSS: {loss_func.__name__}")


In [ ]:
# train and validate
for epoch in range(num_epochs):
        comet_experiment.set_epoch(epoch)

        model.eval()
        with comet_experiment.validate() as validat, torch.no_grad() as nograd:
            for idx, batch in tqdm(enumerate(val_loader), desc=f"VAL_{epoch}"):
                comet_experiment.set_step(idx + epoch * len(val_loader))

                images = batch["image"] / 255.0
                # labels = batch["label"]
                images = images.to(device)
                predictions, latents = model(images)
                loss = loss_func(predictions, images)

                metric = structural_similarity_index_measure(
                    predictions, images)

                comet_experiment.log_metric("loss", loss.item())
                comet_experiment.log_metric("SSIM", metric.item())

                # if idx < 4:
                #     images = einops.rearrange(
                #         [images, predictions],
                #         "source batch 1 height width -> batch height (source width)",
                #     ).cpu()
                #     comet_experiment.log_image(
                #         images[0], f"images_{idx}", step=epoch)

        model.train()
        with comet_experiment.train() as train:
            for idx, batch in tqdm(enumerate(train_loader), desc=f"TRAIN_{epoch}"):
                comet_experiment.set_step(idx + epoch * len(train_loader))

                # look at: model.encoder.parameters().__next__()
                # look at: model.encoder.parameters().__next__().grad
                # look at: latents.shape

                optimizer.zero_grad()  # MUST be called on every batch
                images = batch["image"] / 255.0
                labels = batch["style"]
                images = images.cuda()
                predictions, latents = model(images)
                loss = loss_func(predictions, images)
                loss.backward()
                optimizer.step()

                metric = structural_similarity_index_measure(
                    predictions, images)

                comet_experiment.log_metric("loss", loss.item())
                comet_experiment.log_metric("SSIM", metric.item())

                if not idx % 50:
                    comet_experiment.log_histogram_3d(
                        latents.detach().cpu(), "latents")

VAL_0: 4it [02:14, 32.64s/it]

In [ ]:
model.eval()
with comet_experiment.test() as test, torch.no_grad():
    for idx, batch in tqdm(enumerate(test_loader), desc=f"TEST_{num_epochs}"):
        comet_experiment.set_step(idx + num_epochs * len(test_loader))

        images = batch["image"] / 255.0
        labels = batch["style"]
        images = images.cuda()

        # Model predictions and latents
        predictions, latents = model(images)

        # Compute loss
        loss = loss_func(predictions, images)

        # Compute SSIM metric
        metric = structural_similarity_index_measure(predictions, images)

        # Log metrics
        comet_experiment.log_metric("loss", loss.item())
        comet_experiment.log_metric("SSIM", metric.item())

        # Combine images and predictions for visualization
        combined = torch.stack([images, predictions], dim=0)  # Shape: [2, batch, channels, height, width]
        combined = einops.rearrange(
            combined,
            "source batch channels height width -> (batch source) (height width channels)",
        ).cpu()

        # Log combined image
        comet_experiment.log_image(
            combined, f"images_{idx}", step=num_epochs
        )

In [105]:
# Log model
log_model(comet_experiment, model, model_name="AutoEncoder")

In [2]:
batch = enumerate(test_loader)
images = batch["image"] / 255.0
labels = batch["style"]
images = images.cuda()

# Model predictions and latents
predictions, latents = model(images)

combined = torch.stack([images, predictions], dim=0)  # Shape: [2, batch, channels, height, width]
combined = einops.rearrange(
    combined,
    "source batch channels height width -> (batch source) (height width channels)",
).cpu()

# Log combined image
comet_experiment.log_image(
    combined, f"images_{idx}", step=num_epochs
        )

NameError: name 'test_loader' is not defined